# Day 23: Opening the Turing Lock

[*Advent of Code 2015 day 23*](https://adventofcode.com/2015/day/23) and [*solution megathread*](https://www.reddit.com/3xxdxt)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/master/2015/23/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/master?filepath=2015%2F23%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

...

In [6]:
testdata = []

# inputdata = downloaded['input']

In [ ]:
# assert(my_part1_solution(testdata) == ...)

In [ ]:
# my_part1_solution(inputdata)

In [7]:
HTML(downloaded['part1_footer'])

## Part Two

In [ ]:
# HTML(downloaded['part2'])

In [ ]:
# HTML(downloaded['part2_footer'])

In [ ]:
# assert(my_part2_solution(testdata) == ...)

In [ ]:
# my_part2_solution(inputdata)